Read the following data set:
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/
Rename the columns as per the description from this file:
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
Task:
Create a sql db from adult dataset and name it sqladb
1. Select 10 records from the adult sqladb
2. Show me the average hours per week of all men who are working in private sector
3. Show me the frequency table for education, occupation and relationship, separately
4. Are there any people who are married, working in private sector and having a masters
degree
5. What is the average, minimum and maximum age group for people working in
different sectors
6. Calculate age distribution by country
7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and
'capital-loss'

In [2]:
# Import all the packages
import pandas as pd
import sqlite3 as sqllite

In [3]:
# Read the data from the given URL and see the top 5 records
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data')
df.head(5)

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [4]:
# Rename the columns as per the description.
df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship' ,'race', 'sex' , 'capital-gain', 'capital-loss' , 'hours-per-week', 'native-country','class' ]    
df = df.infer_objects()

#Strip the Object columns.
df_obj = df.select_dtypes(['object'])
df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [5]:
# Print after rename.
df.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [6]:
# Assign the Database and Table Names to local variables.
db_name = 'sqladb.db'
tbl_name = 'adult_names'

In [7]:
# Open Connection to SQLLIte and insert the data to SQL lite Table.
# Create a sql db from adult dataset and name it sqladb
con = sqllite.connect(db_name)  
cur = con.cursor()   

wildcards = ','.join(['?'] * len(df.columns))      
data = [tuple(x) for x in df.values]
 
cur.execute("drop table if exists %s" % tbl_name)
 
col_str = '"' + '","'.join(df.columns) + '"'
cur.execute("create table %s (%s)" % (tbl_name, col_str))
 
cur.executemany("insert into %s values(%s)" % (tbl_name, wildcards), data)
con.commit()                          
 

In [8]:
# 1. Select 10 records from the adult sqladb
df = pd.read_sql_query("SELECT * FROM adult_names LIMIT 10", con)
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
5,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
6,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
7,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
8,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K
9,37,Private,280464,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,80,United-States,>50K


In [9]:
# 2. Show me the average hours per week of all men who are working in private sector 

average = pd.read_sql_query("SELECT CAST ([hours-per-week] as int) as Hours FROM adult_names where sex = 'Male' and workclass ='Private'  ", con)
print(average.mean())

Hours    42.221226
dtype: float64


In [10]:
# 3. Show me the frequency table for education, occupation and relationship, separately.
education = pd.read_sql_query("SELECT education, count(education) as Frequency FROM adult_names GROUP BY education", con)
print(education)

       education  Frequency
0           10th        933
1           11th       1175
2           12th        433
3        1st-4th        168
4        5th-6th        333
5        7th-8th        646
6            9th        514
7     Assoc-acdm       1067
8      Assoc-voc       1382
9      Bachelors       5354
10     Doctorate        413
11       HS-grad      10501
12       Masters       1723
13     Preschool         51
14   Prof-school        576
15  Some-college       7291


In [11]:
occupation = pd.read_sql_query("SELECT occupation, COUNT(occupation) as Frequency FROM adult_names GROUP BY occupation", con)
print(occupation)

           occupation  Frequency
0                   ?       1843
1        Adm-clerical       3769
2        Armed-Forces          9
3        Craft-repair       4099
4     Exec-managerial       4066
5     Farming-fishing        994
6   Handlers-cleaners       1370
7   Machine-op-inspct       2002
8       Other-service       3295
9     Priv-house-serv        149
10     Prof-specialty       4140
11    Protective-serv        649
12              Sales       3650
13       Tech-support        928
14   Transport-moving       1597


In [12]:
relationship = pd.read_sql_query("SELECT relationship, COUNT(relationship) as Frequency FROM adult_names GROUP BY relationship", con)
print(relationship)

     relationship  Frequency
0         Husband      13193
1   Not-in-family       8304
2  Other-relative        981
3       Own-child       5068
4       Unmarried       3446
5            Wife       1568


In [13]:
#4. Are there any people who are married, working in private sector and having a masters degree
people = pd.read_sql_query("SELECT Count(*) as Count FROM adult_names where education = 'Masters' and workclass ='Private' and [marital-status] like 'Married%'  ", con)
people

,Count
0,540


In [14]:
#5. What is the average, minimum and maximum age group for people working in different sectors
agegroup = pd.read_sql_query("SELECT workclass, avg(age) as Average, min(age) as Min, max(age) as Max FROM adult_names group by workclass ", con)
agegroup

,workclass,Average,Min,Max
0,?,40.960240,17,90
1,Federal-gov,42.590625,17,90
2,Local-gov,41.751075,17,90
3,Never-worked,20.571429,17,30
4,Private,36.797585,17,90
5,Self-emp-inc,46.017025,17,84
6,Self-emp-not-inc,44.969697,17,90
7,State-gov,39.436392,17,81
8,Without-pay,47.785714,19,72


In [15]:
#6. Calculate age distribution by country
dfcountry = pd.read_sql_query("SELECT [native-country], age, count(*) as Distribution  FROM adult_names group by  [native-country], age ", con)
dfcountry

,native-country,age,Distribution
0,?,17,2
1,?,18,8
2,?,19,5
3,?,20,10
4,?,21,11
5,?,22,12
6,?,23,6
7,?,24,14
8,?,25,11
9,?,26,18


In [16]:
#7 Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss'
df = pd.read_sql_query("SELECT * FROM adult_names", con)
df['Net-Capital-Gain'] = df['capital-gain' ] - df['capital-loss']
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class,Net-Capital-Gain
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,0
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,0
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,0
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,0
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K,0
5,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K,0
6,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K,0
7,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K,14084
8,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K,5178
9,37,Private,280464,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,80,United-States,>50K,0
